# Tomorrow's Map Generation 

Project for [CDL Quantum Hackathon 2021](https://www.creativedestructionlab.com/streams/quantum/) by the team **The Tomorrow** (Ravindra Babu, Klem Jankiewicz, Piotr Migdał, Victor Onofre, Ekaterina Sokolovskaya).

Source (MIT License): https://github.com/stared/cdl-the-tomorrow

We use discrete optimization for generating two-dimensions grids of tiles. 

We use [D-Wave Ocean SDK](https://docs.ocean.dwavesys.com/en/stable/). Within this notebook you use their quantum devices!

In [ ]:
!pip install dwave-ocean-sdk==3.4.1 --upgrade --quiet

If you want to run it on actual quantum devices (you do!), procive API Token for your D-Wave account. Create path `y` and fill the `Authentication token`; leave other fields with their defaults.

In [ ]:
!dwave config create

In [ ]:
# only locally, won't work on Colab
# !dwave install inspector

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# D-Wave
from dimod import AdjVectorBQM
from neal import SimulatedAnnealingSampler
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
# import dwave.inspector as inspector  # only for Jupyter

Let's define the number of available tiles types and desired size of the output. Both of these values need to be filled now, as they affect number of qubits used.

In [ ]:
n_tiles = 4  #@param {type:"integer", min:2}

size = 8  #@param {type:"integer", min:3}

## Binary quadratic model

We create a [Binary Quadratic Model](https://docs.ocean.dwavesys.com/en/stable/concepts/bqm.html) (BQM). We use binary convention (i.e. 0s and 1s) rather than spin-½ convention (i.e. +1s and -1s).

We have a decision variable, i.e. $\text{tile}_{xy} = k$ for every position $(x, y)$ with $k \in [0, 1, \ldots, \text{n_tiles}-1]$ being an integer code for the tile.

We use [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) - i.e. for each tile we have $k$ binary variables, out of which there is exactly 1 value.

In [ ]:
bqm = AdjVectorBQM('BINARY')

qubit = {(x, y, k): bqm.add_variable((f'tile_{x}_{y}', k), 0)
          for x in range(size)
          for y in range(size)
          for k in range(n_tiles)}

We add constraints ensuring that one-hot encoding is fulfilled.

In [ ]:
one_hot_penalty = 50.  #@param {type:"number"}
for x in range(size):
    for y in range(size):
        bqm.add_linear_equality_constraint(
            [(qubit[x, y, k], 1.0) for k in range(n_tiles)],
            constant=-1.0,
            lagrange_multiplier=one_hot_penalty
        )

We add the crucial part - any costs, weights and constraints for tile adjacency. We can have cost function or hard constraints (i.e. ones we don't want to break).

In this case hard constraints are equivalent to cost = $\infty$. In practice, we use a very high cost, as in `one_hot_penalty`.

We can create costs in any way. It can be arbitrary number, or derived from probabilites. If we start with an example tile set and calculate concidences, to recreate a similar pattern, we **maximize** likelihood, which is a product over all edges:

$$ \text{likelihood} = \prod_{<i,j>} p(t_i, t_j) $$

It is very small, to small to use floating-point number. We logarithm it and multiply by $-1$, so that it becomes a sum we need to **minimize**:

$$ \text{nll} = - \sum_{<i,j>} \log[p(t_i, t_j)] $$

Now we can interpret these a cost terms.



In [ ]:
coincidences =  np.array([[4., 6., 4., 8.],
                           [6., 0., 0., 4.],
                           [4., 0., 0., 0.],
                           [8., 4., 0., 5.]]) 

coincidence_prob = coincidences / coincidences.sum()
coincidence_nll = -np.log(coincidence_prob)

noise = 0.5  #@param {type:"number"}
pair_cost = -np.log((coincidences + noise) / (coincidences + noise).sum())
assert pair_cost.shape == (n_tiles, n_tiles)

# sns.heatmap(pair_cost)

In [ ]:
for i in range(size):
    for j in range(size):
        for k in [i-1, i+1]:
            if ((k<0) or (k>=size))==False:
                for t_i in range(n_tiles):
                    for t_k in range(n_tiles):
                        bqm.add_interaction(qubit[i, j, t_i], qubit[k, j, t_k], pair_cost[t_i][t_k])
        for l in [j-1, j+1]:
            if ((l<0) or (l>=size))==False:
                for t_j in range(n_tiles):
                    for t_l in range(n_tiles):
                        bqm.add_interaction(qubit[i, j, t_j], qubit[i, l, t_l], pair_cost[t_j][t_l])

## Run on a classical simulator

In [ ]:
sampleset = SimulatedAnnealingSampler().sample(bqm, num_reads=100, num_sweeps=10000)
sampleset.to_pandas_dataframe().head()

## Run on a quantum D-Wave device

In [ ]:
sampler = LeapHybridSampler()
sampleset = sampler.sample(bqm)
# sampler = EmbeddingComposite(DWaveSampler())
#sampleset = sampler.sample(bqm,
#                           num_reads=10,
#                           annealing_time=10,
#                           auto_scale=True,
#                           answer_mode='raw',
#                           return_embedding=True)

In [ ]:
# only for local Jupyter Notebook and for EmbeddingComposite(DWaveSampler())
# inspector.show(sampleset)

In [ ]:
sampleset.to_pandas_dataframe()

In [ ]:
# Get the best solution 
best_solution = sampleset.first.sample
assignments = {grid_cell: tile_id for (grid_cell, tile_id), value in best_solution.items() if value}

print(assignments)

In [ ]:
len(assignments)

## Convert the solution to a map

In [ ]:
output_map = np.array(list(assignments.values())).reshape(size, size)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(5,4))

tile_names = ['grass', 'tree', 'water', 'road']
tile_colors  = ['#0c0', '#050', '#00c', '#cc6']
 
sns.heatmap(output_map, cmap=tile_colors, linewidth=0.1, linecolor='black', ax=ax)

colorbar = ax.collections[0].colorbar
M = output_map.max().max()
colorbar.set_ticks([(i + 0.5) * (n_tiles - 1) / n_tiles for i in range(n_tiles)])
colorbar.set_ticklabels(tile_names)

plt.show()

## Thank you!